In [3]:
import ifcopenshell
import pandas as pd
import os
import unicodedata
import csv # Importiamo csv per un controllo più fine del quoting

# --- Mapping IFC Entity Names to Italian Names for CSV filenames ---
IFC_TO_ITALIAN_NAMES_MAP = {
    "IfcBeam": "Travi",
    "IfcBuildingElementProxy": "ElementiGenerici", 
    "IfcColumn": "Pilastri",
    "IfcCovering": "Controsoffitti", # O coperture in generale, a seconda del contesto
    "IfcCurtainWall": "FacciateContinue",
    "IfcDoor": "Porte",
    "IfcWindow": "Finestre",
    "IfcMember": "Membri",
    "IfcOpeningElement": "Aperture",
    "IfcPlate": "Lastre",
    "IfcRailing": "Ringhiere",
    "IfcRoof": "Tetti",
    "IfcSite": "Sito",
    "IfcSlab": "Solai",
    "IfcStair": "Scale",
    "IfcStairFlight": "RampeScale",
    "IfcWall": "Muri",
    "IfcFurniture": "Mobili",
    # Aggiungi qui altre mappature se necessario
    # "IfcSpace": "Spazi",
    # "IfcFooting": "Fondazioni"
}

# --- Funzioni di supporto ---

def normalize_name(name):
    """Normalizza una stringa per renderla case-insensitive e rimuovere spazi/underscore."""
    if name is None:
        return ""
    return unicodedata.normalize('NFKD', str(name)).encode('ascii', 'ignore').decode('utf-8').upper().replace(" ", "").replace("_", "")

def get_property_value(element_or_type, search_prop_name, pset_name_to_search=None):
    """
    Cerca un valore di proprietà in un elemento IFC o nella sua definizione di tipo.
    search_prop_name dovrebbe essere già normalizzato.
    Se pset_name_to_search è specificato, cerca solo in quel Pset.
    Altrimenti, cerca in tutti i Pset.
    """
    if hasattr(element_or_type, "IsDefinedBy"):
        for rel in element_or_type.IsDefinedBy:
            if rel.is_a("IfcRelDefinesByProperties"):
                property_set = rel.RelatingPropertyDefinition
                if property_set and hasattr(property_set, "Name") and hasattr(property_set, "HasProperties"):
                    if pset_name_to_search is not None and normalize_name(property_set.Name) != normalize_name(pset_name_to_search):
                        continue
                    
                    for prop in property_set.HasProperties:
                        normalized_prop_name = normalize_name(prop.Name)
                        if normalized_prop_name == search_prop_name:
                            if prop.is_a("IfcPropertySingleValue"):
                                return prop.NominalValue.wrappedValue if hasattr(prop.NominalValue, 'wrappedValue') else prop.NominalValue
                            elif prop.is_a("IfcPropertyBoundedValue"):
                                return prop.NominalValue.wrappedValue if hasattr(prop.NominalValue, 'wrappedValue') else prop.NominalValue
                            elif prop.is_a("IfcPropertyEnumeratedValue"):
                                if hasattr(prop.EnumerationValues, 'Name'):
                                    return prop.EnumerationValues.Name
                                elif hasattr(prop.EnumerationValues, 'wrappedValue'):
                                    return prop.EnumerationValues.wrappedValue
                                elif prop.EnumerationValues:
                                    return ", ".join([str(v) for v in prop.EnumerationValues])
                                return None
    return None

def get_quantity_value(element_or_type, search_qty_name, qset_name_to_search=None):
    """
    Cerca un valore di quantità in un elemento IFC o nella sua definizione di tipo.
    search_qty_name dovrebbe essere già normalizzato.
    Se qset_name_to_search è specificato, cerca solo in quel Qset.
    Altrimenti, cerca in tutti i Qset.
    """
    if hasattr(element_or_type, "IsDefinedBy"):
        for rel in element_or_type.IsDefinedBy:
            if rel.is_a("IfcRelDefinesByProperties"):
                definition = rel.RelatingPropertyDefinition
                if definition and definition.is_a("IfcElementQuantity") and hasattr(definition, "Name") and hasattr(definition, "Quantities"):
                    if qset_name_to_search is not None and normalize_name(definition.Name) != normalize_name(qset_name_to_search):
                        continue
                    
                    for qty in definition.Quantities:
                        normalized_qty_name = normalize_name(qty.Name)
                        if normalized_qty_name == search_qty_name:
                            if qty.is_a("IfcQuantityArea"):
                                return qty.AreaValue
                            elif qty.is_a("IfcQuantityVolume"):
                                return qty.VolumeValue
                            elif qty.is_a("IfcQuantityLength"):
                                return qty.LengthValue
                            elif qty.is_a("IfcQuantityCount"):
                                return qty.CountValue
    return None

def get_type_definition(element):
    """Trova la definizione di tipo (IfcTypeProduct) per un dato elemento."""
    if hasattr(element, "IsTypedBy"):
        for rel in element.IsTypedBy:
            if rel.is_a("IfcRelDefinesByType") and hasattr(rel, "RelatingType"):
                return rel.RelatingType
    return None

def create_entity_summary_csv(processed_entity_types, output_folder):
    """
    Crea un CSV con un elenco di tutti i tipi di entità IFC che sono stati processati con successo
    (ovvero, per i quali è stato generato un CSV).
    Nota: qui il nome del file rimane basato sull'entità IFC.
    """
    print("\nCreazione del riepilogo delle entità processate...")
    
    if processed_entity_types:
        df_summary = pd.DataFrame(sorted(list(processed_entity_types)), columns=["IFC_EntityType"])
        output_file = os.path.join(output_folder, "IFC_Entity_Summary.csv")
        try:
            df_summary.to_csv(output_file, index=False, encoding='utf-8', sep=';')
            print(f"Riepilogo entità processate salvato in: {output_file}")
        except Exception as e:
            print(f"Errore durante la scrittura del riepilogo entità CSV: {e}")
    else:
        print("Nessun tipo di entità è stato processato per la creazione di CSV.")

def get_unit_conversion_factors(model):
    """
    Determina i fattori di conversione per Lunghezza, Area e Volume in base
    alle unità definite nel modello IFC, convertendoli in metri, metri quadrati e metri cubi.
    Ritorna un dizionario con i fattori di conversione.
    """
    factors = {
        'LENGTH': 1.0, # Default: si assume che sia già in metri
        'AREA': 1.0,   # Default: si assume che sia già in metri quadrati
        'VOLUME': 1.0  # Default: si assume che sia già in metri cubi
    }

    try:
        project = model.by_type("IfcProject")[0]
        if project and hasattr(project, "UnitsInContext"):
            unit_assignment = project.UnitsInContext

            for unit in unit_assignment.Units:
                if unit.is_a("IfcSIUnit"):
                    if unit.UnitType == "LENGTHUNIT":
                        if unit.Prefix == "MILLI": # Millimetri
                            factors['LENGTH'] = 0.001
                        elif unit.Prefix == "CENTI": # Centimetri
                            factors['LENGTH'] = 0.01
                        elif unit.Prefix == "DECI": # Decimetri
                            factors['LENGTH'] = 0.1
                        elif unit.Prefix == "KILO": # Chilometri
                            factors['LENGTH'] = 1000.0
                        
                        factors['AREA'] = factors['LENGTH'] ** 2
                        factors['VOLUME'] = factors['LENGTH'] ** 3
                        
                    elif unit.UnitType == "AREAUNIT" and unit.Name == "SQUARE_METRE" and not unit.Prefix:
                        factors['AREA'] = 1.0 
                    elif unit.UnitType == "VOLUMEUNIT" and unit.Name == "CUBIC_METRE" and not unit.Prefix:
                        factors['VOLUME'] = 1.0 

                elif unit.is_a("IfcConversionBasedUnit"):
                    if hasattr(unit, 'UnitType') and hasattr(unit, 'Name') and hasattr(unit, 'ConversionFactor') and hasattr(unit, 'BaseUnit'):
                        if unit.BaseUnit.is_a("IfcSIUnit") and unit.BaseUnit.UnitType == "LENGTHUNIT" and unit.BaseUnit.Name == "METRE":
                            if unit.UnitType == "LENGTHUNIT":
                                factors['LENGTH'] = unit.ConversionFactor
                                factors['AREA'] = factors['LENGTH'] ** 2
                                factors['VOLUME'] = factors['LENGTH'] ** 3
    except IndexError:
        print("Attenzione: Nessun IfcProject trovato nel modello per determinare le unità. Verranno usate le unità di default (metri, mq, mc).")
    except Exception as e:
        print(f"Attenzione: Errore durante la determinazione delle unità del modello: {e}. Verranno usate le unità di default (metri, mq, mc).")

    print(f"Fattori di conversione delle unità (in base al modello): {factors}")
    return factors


# --- Funzione principale di estrazione dati (modificata per TagCode e unità) ---
def extract_ifc_data_for_entity_type(model, entity_type, properties_to_extract, unit_factors, output_folder):
    """
    Estrae dati per un tipo di entità specifico e li salva in un file CSV,
    applicando i fattori di conversione delle unità.
    """
    print(f"Elaborazione di: {entity_type}...")
    elements = model.by_type(entity_type) 
    
    if not elements:
        print(f"Nessun elemento di tipo {entity_type} trovato nel file IFC.")
        return False

    data_for_csv = []

    for element in elements:
        row_data = {
            "ifcclass": element.is_a(),
            "GlobalId": element.GlobalId,
            "typename": getattr(element, "ObjectType", ""), 
            "NAME": element.Name if element.Name else "",
        }
        
        element_type_definition = get_type_definition(element)

        # --- Estrazione TAG-CODE (modificato per essere sempre stringa) ---
        tag_val = getattr(element, "Tag", None)
        if tag_val is None:
            tag_val = get_property_value(element, normalize_name("TAG-CODE")) or \
                      get_property_value(element, normalize_name("Tag")) or \
                      get_property_value(element, normalize_name("Mark"))
        if tag_val is None and element_type_definition:
            tag_val = get_property_value(element_type_definition, normalize_name("CR_TAG_CODE")) or \
                      get_property_value(element_type_definition, normalize_name("CR_TAG_CODE_de")) or \
                      get_property_value(element_type_definition, normalize_name("Tag")) or \
                      get_property_value(element_type_definition, normalize_name("Mark"))
        
        # Converte sempre il TagCode in stringa. Se None, diventa stringa vuota.
        row_data["TAG-CODE"] = str(tag_val) if tag_val is not None else ""
        
        # --- Estrazione Livello (CR_WBS_LIVELLO) ---
        level_name = None
        for rel in getattr(element, "ContainedInStructure", []):
            if hasattr(rel, "RelatingStructure") and hasattr(rel.RelatingStructure, "Name"):
                level_name = rel.RelatingStructure.Name
                break
        row_data["CR_WBS_LIVELLO"] = level_name
        
        # --- Estrazione Materiale come LISTA ---
        element_materials_list = [] 
        
        def extract_materials(obj):
            materials = []
            if hasattr(obj, "HasAssociations"):
                for assoc in obj.HasAssociations:
                    if assoc.is_a("IfcRelAssociatesMaterial"):
                        mat = assoc.RelatingMaterial
                        if mat:
                            if mat.is_a("IfcMaterial"):
                                materials.append(mat.Name if mat.Name is not None else "")
                            elif hasattr(mat, "ForLayerSet") and hasattr(mat.ForLayerSet, "MaterialLayers"):
                                for layer in mat.ForLayerSet.MaterialLayers:
                                    if hasattr(layer, "Material") and layer.Material is not None and hasattr(layer.Material, "Name"):
                                        materials.append(layer.Material.Name if layer.Material.Name is not None else "")
                                    else:
                                        materials.append("") 
                            elif hasattr(mat, "MaterialConstituents") and hasattr(mat, "MaterialConstituents"): 
                                for constituent in mat.MaterialConstituents:
                                    if hasattr(constituent, "Material") and constituent.Material is not None and hasattr(constituent.Material, "Name"):
                                        materials.append(constituent.Material.Name if constituent.Material.Name is not None else "")
                                    else:
                                        materials.append("")
                        break 
            return materials

        element_materials_list = extract_materials(element)
        
        if not element_materials_list and element_type_definition:
            element_materials_list = extract_materials(element_type_definition)
        
        row_data["_MATERIALS_LIST_TEMP"] = element_materials_list 


        # Estrazione delle proprietà specifiche richieste
        for prop_info in properties_to_extract:
            prop_csv_name = prop_info["name"] 
            source = prop_info["source"]
            ifc_prop_name_normalized = normalize_name(prop_info.get("ifc_name", prop_csv_name))
            
            value = None
            
            if source == 'attribute':
                value = getattr(element, prop_info.get("ifc_name", prop_csv_name), None)
                if value is None and element_type_definition:
                    value = getattr(element_type_definition, prop_info.get("ifc_name", prop_csv_name), None)
                if hasattr(value, 'wrappedValue'): 
                    value = value.wrappedValue
            elif source == 'pset_any_name':
                value = get_property_value(element, ifc_prop_name_normalized, pset_name_to_search=None) 
                if value is None and element_type_definition:
                    value = get_property_value(element_type_definition, ifc_prop_name_normalized, pset_name_to_search=None)
            elif source == 'quantity_any_name': 
                value = get_quantity_value(element, ifc_prop_name_normalized, qset_name_to_search=None) 
                if value is None and element_type_definition:
                    value = get_quantity_value(element_type_definition, ifc_prop_name_normalized, qset_name_to_search=None)
            elif source == 'pset': 
                pset_name = prop_info.get("pset_qset_name")
                value = get_property_value(element, ifc_prop_name_normalized, pset_name_to_search=pset_name)
                if value is None and element_type_definition:
                    value = get_property_value(element_type_definition, ifc_prop_name_normalized, pset_name_to_search=pset_name)
            elif source == 'quantity': 
                qset_name = prop_info.get("pset_qset_name")
                value = get_quantity_value(element, ifc_prop_name_normalized, qset_name_to_search=qset_name)
                if value is None and element_type_definition:
                    value = get_quantity_value(element_type_definition, ifc_prop_name_normalized, qset_name_to_search=qset_name)

            # --- Applicazione dei fattori di conversione ---
            if isinstance(value, (int, float)):
                if prop_csv_name == "AREA":
                    value *= unit_factors['AREA']
                elif prop_csv_name == "VOLUME":
                    value *= unit_factors['VOLUME']
                elif prop_csv_name == "LUNGHEZZA":
                    value *= unit_factors['LENGTH']
                # Applica anche a OverallHeight e OverallWidth se sono numerici
                elif prop_csv_name in ["OverallHeight", "OverallWidth"]:
                    value *= unit_factors['LENGTH']
                
                value = f"{value:.3f}".replace('.', ',') # Formattazione con virgola decimale
            
            row_data[prop_csv_name] = value

        # --- Logica di calcolo AREA solo per IfcDoor e IfcWindow (modificata per usare i valori già convertiti) ---
        if entity_type in ["IfcDoor", "IfcWindow"]:
            overall_height_converted = row_data.get("OverallHeight")
            overall_width_converted = row_data.get("OverallWidth")
            
            try:
                # Convertiamo da stringa (con virgola) a float per il calcolo
                height_calc = float(str(overall_height_converted).replace(',', '.')) if isinstance(overall_height_converted, str) else overall_height_converted
                width_calc = float(str(overall_width_converted).replace(',', '.')) if isinstance(overall_width_converted, str) else overall_width_converted
            except (ValueError, TypeError):
                height_calc = None
                width_calc = None

            if height_calc is not None and width_calc is not None:
                try:
                    calculated_area = height_calc * width_calc
                    row_data["AREA"] = f"{calculated_area:.3f}".replace('.', ',')
                except (ValueError, TypeError):
                    row_data["AREA"] = None
            else:
                row_data["AREA"] = None
        
        data_for_csv.append(row_data)

    df = pd.DataFrame(data_for_csv)

    # --- LOGICA: ESPANSIONE DEGLI STRATI DI MATERIALE ---
    if "_MATERIALS_LIST_TEMP" in df.columns and any(isinstance(x, list) and x for x in df["_MATERIALS_LIST_TEMP"]):
        max_layers = 0
        for item_list in df["_MATERIALS_LIST_TEMP"]:
            if isinstance(item_list, list):
                max_layers = max(max_layers, len(item_list))
        
        if max_layers > 0:
            material_layers_df = df['_MATERIALS_LIST_TEMP'].apply(pd.Series).add_prefix('Strato ')
            df = pd.concat([df, material_layers_df], axis=1)
            
        df = df.drop(columns=["_MATERIALS_LIST_TEMP"])
    else:
        if "_MATERIALS_LIST_TEMP" in df.columns:
            df = df.drop(columns=["_MATERIALS_LIST_TEMP"])

    # --- DETERMINA IL NOME DEL FILE CSV IN ITALIANO ---
    italian_file_name = IFC_TO_ITALIAN_NAMES_MAP.get(entity_type, entity_type) # Ottieni nome italiano o usa quello IFC
    output_file = os.path.join(output_folder, f"{italian_file_name}.csv")

    try:
        fixed_cols_order = ["ifcclass", "GlobalId", "typename", "NAME", "TAG-CODE", "CR_WBS_LIVELLO"]
        
        additional_cols_order = []
        if entity_type in ["IfcDoor", "IfcWindow"]:
            additional_cols_order = ["OverallHeight", "OverallWidth", "AREA", "RESISTENZA AL FUOCO"]
        
        final_columns = []
        for col in fixed_cols_order + additional_cols_order:
            if col in df.columns and col not in final_columns:
                final_columns.append(col)
        
        strato_cols = sorted([col for col in df.columns if col.startswith("Strato ")], key=lambda x: int(x.split(' ')[1]))
        final_columns.extend(strato_cols)

        for col in df.columns:
            if col not in final_columns:
                final_columns.append(col)

        df.to_csv(output_file, index=False, encoding='utf-8', sep=';', columns=final_columns, quoting=csv.QUOTE_MINIMAL)
        print(f"Dati per {entity_type} salvati in: {output_file}")
        return True
    except Exception as e:
        print(f"Errore durante la scrittura del CSV per {entity_type}: {e}")
        return False


# --- Configurazione dei percorsi e chiamata principale ---
# ASSICURATI DI AGGIORNARE QUESTI PERCORSI CON I TUOI REALI PERCORSI!
INPUT_FOLDER = r"C:\Users\filippide.beltrame\OneDrive - Gruppo Carron\Desktop\ifc to json per entity\input"
OUTPUT_FOLDER = r"C:\Users\filippide.beltrame\OneDrive - Gruppo Carron\Desktop\ifc to json per entity\output"

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

ifc_file_found = None
for file_name in os.listdir(INPUT_FOLDER):
    if file_name.lower().endswith(".ifc"):
        ifc_file_found = os.path.join(INPUT_FOLDER, file_name)
        break

if ifc_file_found is None:
    print(f"Errore: Nessun file IFC trovato nella cartella: {INPUT_FOLDER}")
else:
    print(f"Trovato file IFC: {ifc_file_found}")
    try:
        model = ifcopenshell.open(ifc_file_found)
        print("Modello IFC caricato con successo.")
    except Exception as e:
        print(f"Errore durante il caricamento del file IFC: {e}")
        model = None

    if model:
        # --- Ottieni i fattori di conversione una volta all'inizio ---
        unit_factors = get_unit_conversion_factors(model)
        # --- Fine recupero fattori ---

        # Lista delle istanze IFC che si desidera esportare
        ALLOWED_ENTITY_TYPES = [
            "IfcBeam",
            "IfcBuildingElementProxy",
            "IfcColumn",
            "IfcCovering",
            "IfcCurtainWall",
            "IfcDoor",
            "IfcMember",
            "IfcOpeningElement",
            "IfcPlate",
            "IfcRailing",
            "IfcRoof",
            "IfcSite",
            "IfcSlab",
            "IfcStair",
            "IfcStairFlight",
            "IfcWall",
            "IfcFurniture",
            "IfcWindow" 
        ]

        # Dizionario delle definizioni di proprietà specifiche/speciali
        specific_entity_definitions = {
            "IfcDoor": [
                {"name": "OverallHeight", "source": "attribute", "ifc_name": "OverallHeight"},
                {"name": "OverallWidth", "source": "attribute", "ifc_name": "OverallWidth"},
                {"name": "RESISTENZA AL FUOCO", "source": "pset_any_name", "ifc_name": "Resistenza al Fuoco"},
                {"name": "AREA", "source": "pset_any_name", "ifc_name": "Area"},
                {"name": "VOLUME", "source": "pset_any_name", "ifc_name": "Volume"},
            ],
            "IfcWindow": [ 
                {"name": "OverallHeight", "source": "attribute", "ifc_name": "OverallHeight"},
                {"name": "OverallWidth", "source": "attribute", "ifc_name": "OverallWidth"},
                {"name": "RESISTENZA AL FUOCO", "source": "pset_any_name", "ifc_name": "Resistenza al Fuoco"},
                {"name": "AREA", "source": "pset_any_name", "ifc_name": "Area"}, 
                {"name": "VOLUME", "source": "pset_any_name", "ifc_name": "Volume"},
            ],
            "IfcBuildingElementProxy": [
                {"name": "AREA", "source": "pset_any_name", "ifc_name": "Area"}, 
                {"name": "VOLUME", "source": "pset_any_name", "ifc_name": "Volume"},
            ],
            "IfcCovering": [
                {"name": "AREA", "source": "pset_any_name", "ifc_name": "Area"},
                {"name": "VOLUME", "source": "pset_any_name", "ifc_name": "Volume"},
                {"name": "RESISTENZA AL FUOCO", "source": "pset_any_name", "ifc_name": "Resistenza al Fuoco"},
            ],
            "IfcCurtainWall": [
                {"name": "LUNGHEZZA", "source": "pset_any_name", "ifc_name": "Lunghezza"},
                {"name": "AREA", "source": "pset_any_name", "ifc_name": "Area"},
                {"name": "RESISTENZA AL FUOCO", "source": "pset_any_name", "ifc_name": "Resistenza al Fuoco"},
            ],
            "IfcOpeningElement": [
                {"name": "AREA", "source": "pset_any_name", "ifc_name": "Area"},
                {"name": "VOLUME", "source": "pset_any_name", "ifc_name": "Volume"},
                {"name": "RESISTENZA AL FUOCO", "source": "pset_any_name", "ifc_name": "Resistenza al Fuoco"},
            ],
            "IfcPlate": [
                {"name": "AREA", "source": "pset_any_name", "ifc_name": "Area"},
                {"name": "VOLUME", "source": "pset_any_name", "ifc_name": "Volume"},
            ],
            "IfcRoof": [
                {"name": "AREA", "source": "pset_any_name", "ifc_name": "Area"},
                {"name": "VOLUME", "source": "pset_any_name", "ifc_name": "Volume"},
            ],
            "IfcSlab": [
                {"name": "AREA", "source": "pset_any_name", "ifc_name": "Area"},
                {"name": "VOLUME", "source": "pset_any_name", "ifc_name": "Volume"},
            ],
            "IfcStair": [
                # Nessuna proprietà specifica oltre a quelle di default
            ],
            "IfcWall": [
                {"name": "LUNGHEZZA", "source": "pset_any_name", "ifc_name": "Lunghezza"},
                {"name": "AREA", "source": "pset_any_name", "ifc_name": "Area"},
                {"name": "VOLUME", "source": "pset_any_name", "ifc_name": "Volume"},
                {"name": "RESISTENZA AL FUOCO", "source": "pset_any_name", "ifc_name": "Resistenza al Fuoco"},
            ],
        }

        # Definizioni di proprietà di default da applicare a TUTTI i tipi di entità che non hanno definizioni specifiche sopra
        default_properties_to_extract = [
            {"name": "AREA", "source": "pset_any_name", "ifc_name": "Area"},
            {"name": "VOLUME", "source": "pset_any_name", "ifc_name": "Volume"},
            {"name": "LUNGHEZZA", "source": "pset_any_name", "ifc_name": "Lunghezza"},
            {"name": "RESISTENZA AL FUOCO", "source": "pset_any_name", "ifc_name": "Resistenza al Fuoco"},
        ]

        # Ottieni tutti i tipi di entità presenti nel modello che NON sono "Type" (es. IfcWallType)
        all_ifc_types_in_model = set()
        for entity in model.by_type("IfcRoot"):
            entity_name = entity.is_a()
            if not entity_name.endswith("Type"):
                all_ifc_types_in_model.add(entity_name)

        processed_entity_types_for_summary = set()

        # Filtra i tipi di entità da processare solo a quelli nella lista ALLOWED_ENTITY_TYPES
        # e che sono effettivamente presenti nel modello.
        entity_types_to_process = sorted(list(all_ifc_types_in_model.intersection(ALLOWED_ENTITY_TYPES)))

        if not entity_types_to_process:
            print("Attenzione: Nessun tipo di entità specificato trovato nel modello da elaborare.")
        
        for entity_type in entity_types_to_process: 
            properties_list_for_this_type = []
            
            if entity_type in specific_entity_definitions:
                properties_list_for_this_type = specific_entity_definitions[entity_type]
            else:
                properties_list_for_this_type = default_properties_to_extract
            
            # Passa i fattori di conversione alla funzione di estrazione
            if extract_ifc_data_for_entity_type(model, entity_type, properties_list_for_this_type, unit_factors, OUTPUT_FOLDER):
                processed_entity_types_for_summary.add(entity_type)
        
        create_entity_summary_csv(processed_entity_types_for_summary, OUTPUT_FOLDER)

    print("\nProcesso di estrazione dati completato.")

Trovato file IFC: C:\Users\filippide.beltrame\OneDrive - Gruppo Carron\Desktop\ifc to json per entity\input\TE-CR-PC-A0-ARC-M3-110.ifc
Modello IFC caricato con successo.
Fattori di conversione delle unità (in base al modello): {'LENGTH': 0.001, 'AREA': 1.0, 'VOLUME': 1.0}
Elaborazione di: IfcBeam...
Dati per IfcBeam salvati in: C:\Users\filippide.beltrame\OneDrive - Gruppo Carron\Desktop\ifc to json per entity\output\Travi.csv
Elaborazione di: IfcBuildingElementProxy...
Dati per IfcBuildingElementProxy salvati in: C:\Users\filippide.beltrame\OneDrive - Gruppo Carron\Desktop\ifc to json per entity\output\ElementiGenerici.csv
Elaborazione di: IfcColumn...
Dati per IfcColumn salvati in: C:\Users\filippide.beltrame\OneDrive - Gruppo Carron\Desktop\ifc to json per entity\output\Pilastri.csv
Elaborazione di: IfcCovering...
Dati per IfcCovering salvati in: C:\Users\filippide.beltrame\OneDrive - Gruppo Carron\Desktop\ifc to json per entity\output\Controsoffitti.csv
Elaborazione di: IfcCurtainW